In [7]:
%%writefile mapper_total.py
#!/usr/bin/env python2
import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print "%d" % (1)

Writing mapper_total.py


In [8]:
%%writefile reducer_total.py
#!/usr/bin/env python2
import sys

total = 0

for line in sys.stdin:
    total += 1

print "%s\t%d" % ('total', total)

Writing reducer_total.py


In [9]:
%%writefile mapper_freq.py
#!/usr/bin/env python2
import sys
import re

def isWordName(word):
    if len(word) == 0 or word[0].isdigit() or (not word[0].isupper()):
        return False
    else:
        if len(word) > 1:
            for c in word[1:]:
                if c.isupper():
                    return False
        return True

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        if isWordName(word):
            print "%s\t%d" % (word.lower(), 1)

Overwriting mapper_freq.py


In [10]:
%%writefile reducer_freq.py
#!/usr/bin/env python2
import sys

current_key = None
word_sum = 0
total = 0

with open('part-00000') as f:
    tmp = f.readline()
total = int(tmp.strip().split('\t', 1)[1])

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            freq = (word_sum / (total * 1.0)) * 100
            if freq < 0.5:
                print "%s\t%d\t%f" % (current_key, word_sum, freq)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    freq = (word_sum / (total * 1.0)) * 100
    if freq < 0.5:
        print "%s\t%d\t%f" % (current_key, word_sum, freq)

Writing reducer_freq.py


In [3]:
! chmod +x ./mapper_freq.py

In [2]:
! hdfs dfs -copyToLocal /data/wiki/en_articles_part/articles-part

In [23]:
! cat freq_part-00000 | sort -k2nr | cat > my_output.txt

In [5]:
! cat freq_part-00000 | ./mapper_freq.py >> my_output.txt

# Job runner

In [12]:
%%bash
TOTAL_DIR="total"
FREQ_DIR="freq"
OUT_DIR="wordcount_stopwords_result"

rm part-00000
hdfs dfs -rm -r -skipTrash ${TOTAL_DIR} > /dev/null
hdfs dfs -rm -r -skipTrash ${FREQ_DIR} > /dev/null
hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.job.name="Total_count" \
    -D mapreduce.job.reduces=1 \
    -files mapper_total.py,reducer_total.py \
    -mapper "python mapper_total.py" \
    -reducer "python reducer_total.py" \
    -input /data/wiki/en_articles_part \
    -output ${TOTAL_DIR} > /dev/null

hdfs dfs -copyToLocal ./total/part-00000 .

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.job.name="Frequencies_of_names" \
    -D mapreduce.job.reduces=1 \
    -files mapper_freq.py,reducer_freq.py,part-00000 \
    -mapper "python mapper_freq.py" \
    -reducer "python reducer_freq.py" \
    -input /data/wiki/en_articles_part \
    -output ${FREQ_DIR} > /dev/null    
    
yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.job.name="Frequencies_of_names" \
    -D mapreduce.job.reduces=1 \
    -D stream.num.map.output.key.fields=2 \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
    -D mapreduce.partition.keycomparator.options="-k2nr" \
    -files mapper_freq.py,reducer_freq.py,part-00000 \
    -mapper "cat" \
    -reducer "cat" \
    -input ${FREQ_DIR}/part-* \
    -output ${OUT_DIR} > /dev/null        

hdfs dfs -cat ${FREQ_DIR}/part-0000* | head  

a	18574	0.155595
a$(10	1	0.000008
a$1	1	0.000008
a$1.18	1	0.000008
a$1.7	1	0.000008
a$1.95	1	0.000008
a$1400	1	0.000008
a$2	1	0.000008
a$2.3	1	0.000008
a$200	1	0.000008


rm: `wordcount_stopwords_result': No such file or directory
17/11/14 05:35:58 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/11/14 05:35:58 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/11/14 05:35:59 INFO mapred.FileInputFormat: Total input files to process : 1
17/11/14 05:35:59 WARN hdfs.DataStreamer: Caught exception
java.lang.InterruptedException
	at java.lang.Object.wait(Native Method)
	at java.lang.Thread.join(Thread.java:1252)
	at java.lang.Thread.join(Thread.java:1326)
	at org.apache.hadoop.hdfs.DataStreamer.closeResponder(DataStreamer.java:927)
	at org.apache.hadoop.hdfs.DataStreamer.endBlock(DataStreamer.java:578)
	at org.apache.hadoop.hdfs.DataStreamer.run(DataStreamer.java:755)
17/11/14 05:35:59 INFO mapreduce.JobSubmitter: number of splits:2
17/11/14 05:35:59 INFO Configuration.deprecation: mapred.job.name is deprecated. Instead, use mapreduce.job.name
17/11/14 05:36:00 INFO mapreduce.JobSubmitter: Submitting tokens for job:

In [30]:
!hdfs dfs -copyToLocal ./total/part-00000 .

copyToLocal: `part-00000': File exists


In [16]:
!hdfs dfs -copyToLocal freq/part-00000 freq_part-00000

In [17]:
! ls -l

total 120456
-rw-r--r-- 1 jovyan users 76861985 Nov 14 05:16 articles-part
-rw-rw-r-- 1 jovyan root      1634 Oct  9 00:28 Demo.ipynb
-rw-r--r-- 1 jovyan users  3577563 Nov 14 05:41 freq_part-00000
-rwxr-xr-x 1 jovyan users      687 Nov 14 05:21 mapper_freq.py
-rw-r--r-- 1 jovyan users      369 Nov 14 05:21 mapper_total.py
-rw-r--r-- 1 jovyan users 42835716 Nov 14 05:18 my_output.txt
-rw-r--r-- 1 jovyan users    20928 Nov 14 05:41 name_rating (1).ipynb
-rw-r--r-- 1 jovyan users       15 Nov 14 05:36 part-00000
-rw-rw-r-- 1 jovyan root       239 Oct 15 16:10 README.md
-rw-r--r-- 1 jovyan users      732 Nov 14 05:21 reducer_freq.py
-rw-r--r-- 1 jovyan users      118 Nov 14 05:21 reducer_total.py
-rw-r--r-- 1 root   users      757 Nov 14 05:03 supervisord.log
-rw-r--r-- 1 root   users        2 Nov 14 05:03 supervisord.pid
